<a href="https://colab.research.google.com/github/ashwin-yedte/visual-intelligence-travel-finance/blob/main/recommended_destinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VLM INTELLIGENCE LAYER **


STEP 4: RECOMMENDED DESTINATIONS
Enrich top destinations with full metadata for visual gallery display

Features:
- Load top-ranked destinations from Step 3
- Enrich with VL encoding prompts (characteristics)
- Add geo-location data
- Add offers (hotels, activities, flights, packages)
- Prepare gallery-ready JSON for frontend

# =================================================================
 PREREQUISITES
# =================================================================

MUST RUN BEFORE THIS:
1. Step 1 (image analysis)
2. Step 2 (destination matching)
3. Step 3 (theme aggregation and ranking)
   This creates step3_refined_ranking.json


# =================================================================
Step 1: CONFIGURATION
# =================================================================

In [ ]:
print("="*80)
print("STEP 4: RECOMMENDED DESTINATIONS - VLM INTELLIGENCE LAYER")
print("="*80)

class Step4Config:
    """Configuration for Step 4"""

    # Paths
    BASE_PATH = '/content/drive/MyDrive/visual-intelligence-travel-finance'
    PROMPTS_PATH = BASE_PATH + '/data/vl_encoding/prompts'
    METADATA_PATH = BASE_PATH + '/data/landmarks/metadata.json'
    IMAGES_BASE_PATH = BASE_PATH + '/data/landmarks/images'

    # Display settings
    TOP_N_DISPLAY = 10
    MAX_PROMPTS_PER_CATEGORY = 3

    # Output
    OUTPUT_FILE = "step4_recommendations.json"

print("Configuration loaded")
print("="*80)


STEP 4: RECOMMENDED DESTINATIONS - VLM INTELLIGENCE LAYER
Configuration loaded


# =================================================================
Step 2: IMPORT LIBRARY
# =================================================================


In [ ]:
import json
import os
from typing import Dict, List, Any

print("Imports complete")
print("="*80)

Imports complete


# =================================================================
Step 3: LOAD REQUIRED DATA
# =================================================================


In [ ]:
print("\n" + "="*80)
print("LOADING REQUIRED DATA")
print("="*80)
# Load destination prompts
print("Loading destination prompts...")
with open(Step4Config.PROMPTS_PATH + '/destination_prompts.json', 'r') as f:
    destination_prompts = json.load(f)
print("Loaded prompts for " + str(len(destination_prompts)) + " destinations")

# Load metadata
print("\nLoading metadata...")
with open(Step4Config.METADATA_PATH, 'r') as f:
    metadata = json.load(f)
print("Metadata loaded: " + str(metadata['total_destinations']) + " destinations")

print("="*80)



LOADING REQUIRED DATA
Loading destination prompts...


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/prompts/destination_prompts.json'

# =================================================================
Step 4: METADATA ENRICHMENT FUNCTIONS
# =================================================================


In [ ]:
def get_full_destination_metadata(dest_id: str) -> Dict[str, Any]:
    """
    Get complete destination metadata including offers and geo-location.

    Args:
        dest_id: Destination ID

    Returns:
        Full metadata dictionary
    """

    for theme in metadata['themes']:
        for state in theme['states']:
            for destination in state['destinations']:
                if destination['destination_id'] == dest_id:
                    return {
                        'destination_id': dest_id,
                        'destination_name': destination['destination_name'],
                        'state': state['state_name'],
                        'theme': theme['theme_name'],
                        'folder': destination.get('folder', ''),
                        'images': destination.get('images', []),
                        'image_count': destination.get('image_count', 0),
                        'geo_location': destination.get('geo_location', {}),
                        'offers': destination.get('offers', {})
                    }

    return None


def get_destination_characteristics(dest_id: str) -> Dict[str, List[str]]:
    """
    Get semantic characteristics from VL encoding prompts.

    Args:
        dest_id: Destination ID

    Returns:
        Dictionary of category: [prompts]
    """

    if dest_id not in destination_prompts:
        return {}

    dest_data = destination_prompts[dest_id]
    aggregated = dest_data.get('aggregated_prompts', {})

    characteristics = {}

    for category, prompts in aggregated.items():
        # Get top N prompts by weighted score
        sorted_prompts = sorted(
            prompts,
            key=lambda x: x.get('weighted_score', 0),
            reverse=True
        )

        # Extract just the text
        characteristics[category] = [
            p['text'] for p in sorted_prompts[:Step4Config.MAX_PROMPTS_PER_CATEGORY]
        ]

    return characteristics


def get_image_urls(dest_id: str, folder: str, images: List[str]) -> List[str]:
    """
    Generate image URLs/paths for gallery display.

    Args:
        dest_id: Destination ID
        folder: Image folder path
        images: List of image filenames

    Returns:
        List of image paths
    """

    if not images:
        return []

    # In production, these would be actual URLs
    # For now, return Drive paths
    image_paths = []
    for img_filename in images[:5]:
        path = Step4Config.IMAGES_BASE_PATH + '/' + folder + '/' + img_filename
        image_paths.append(path)

    return image_paths


print("Enrichment functions loaded")
print("="*80)


Enrichment functions loaded


# =================================================================
Step 5: BUILD RECOMMENDATIONS
# =================================================================


In [ ]:
def build_recommendations(ranked_destinations: List[Dict], theme_analysis: Dict) -> Dict[str, Any]:
    """
    Build complete recommendation package with all metadata.

    Args:
        ranked_destinations: Top-ranked destinations from Step 3
        theme_analysis: Theme analysis from Step 3

    Returns:
        Complete recommendations dictionary
    """

    print("\n" + "="*80)
    print("BUILDING RECOMMENDATIONS")
    print("="*80)

    recommendations = []

    for i, dest in enumerate(ranked_destinations[:Step4Config.TOP_N_DISPLAY], 1):
        dest_id = dest['destination_id']

        print("\nEnriching " + str(i) + ". " + dest['destination_name'])

        # Get full metadata
        full_metadata = get_full_destination_metadata(dest_id)

        if not full_metadata:
            print("  WARNING: Metadata not found, skipping")
            continue

        # Get characteristics
        characteristics = get_destination_characteristics(dest_id)
        print("  Characteristics: " + str(len(characteristics)) + " categories")

        # Get image URLs
        image_urls = get_image_urls(
            dest_id,
            full_metadata['folder'],
            full_metadata['images']
        )
        print("  Images: " + str(len(image_urls)))

        # Build recommendation object
        recommendation = {
            'rank': i,
            'destination_id': dest_id,
            'destination_name': dest['destination_name'],
            'state': dest['state'],
            'theme': dest['theme'],
            'similarity_score': dest['avg_similarity'],
            'max_similarity': dest['max_similarity'],
            'appearances': dest['appearances'],
            'appeared_in_images': dest['appeared_in_images'],
            'theme_match': dest['theme_match'],
            'final_score': dest['final_score'],
            'characteristics': characteristics,
            'geo_location': full_metadata['geo_location'],
            'offers': full_metadata['offers'],
            'images': image_urls,
            'image_count': len(image_urls)
        }

        recommendations.append(recommendation)

    print("\n" + "="*80)
    print("ENRICHMENT COMPLETE")
    print("="*80)
    print("Total recommendations: " + str(len(recommendations)))
    print("="*80)

    return {
        'user_profile': {
            'dominant_theme': theme_analysis['dominant_theme'],
            'theme_confidence': round(theme_analysis['theme_confidence'] * 100, 2),
            'theme_distribution': theme_analysis['theme_distribution']
        },
        'recommendations': recommendations,
        'total_recommendations': len(recommendations)
    }


print("Build recommendations function loaded")
print("="*80)


Build recommendations function loaded


# =================================================================
Step 6: SAVE STEP 4 OUTPUTS
# =================================================================


In [ ]:
def save_step4_outputs(recommendations_data: Dict) -> None:
    """
    Save final recommendations for frontend display.

    Saves:
    - step4_recommendations.json: Complete gallery data
    """

    print("\n" + "="*80)
    print("SAVING STEP 4 OUTPUTS")
    print("="*80)

    output_path = '/content/' + Step4Config.OUTPUT_FILE
    with open(output_path, 'w') as f:
        json.dump(recommendations_data, f, indent=2)

    print("Saved to: " + output_path)
    print("="*80)
    print("\nRECOMMENDATIONS READY FOR DISPLAY")
    print("="*80)


print("Save function loaded")
print("="*80)


Save function loaded


# =================================================================
Step 7: MAIN EXECUTION FUNCTION
# =================================================================


In [ ]:
def run_step4(step3_output_path: str = '/content/step3_refined_ranking.json'):
    """
    Complete Step 4 execution.

    Args:
        step3_output_path: Path to Step 3 output file

    Returns:
        Complete recommendations dictionary
    """

    print("\n" + "="*80)
    print("EXECUTING STEP 4: RECOMMENDED DESTINATIONS")
    print("="*80)

    # Load Step 3 results
    print("\nLoading Step 3 results...")
    try:
        with open(step3_output_path, 'r') as f:
            step3_data = json.load(f)
        print("Loaded Step 3 results")
    except FileNotFoundError:
        print("ERROR: Step 3 results not found at " + step3_output_path)
        print("Please run Step 3 first")
        return None

    ranked_destinations = step3_data['ranked_destinations']
    theme_analysis = step3_data['theme_analysis']

    # Build recommendations
    recommendations_data = build_recommendations(ranked_destinations, theme_analysis)

    # Save outputs
    save_step4_outputs(recommendations_data)

    print("\n" + "="*80)
    print("STEP 4 COMPLETE")
    print("="*80)
    print("\nSummary:")
    print("  Theme: " + recommendations_data['user_profile']['dominant_theme'])
    print("  Confidence: " + str(recommendations_data['user_profile']['theme_confidence']) + "%")
    print("  Recommendations: " + str(recommendations_data['total_recommendations']))
    print("  Top destination: " + recommendations_data['recommendations'][0]['destination_name'])
    print("="*80)

    return recommendations_data


print("Main execution function loaded")
print("="*80)
print("\nSTEP 4 INITIALIZED - Ready to build recommendations")
print("="*80)
print("\nTO RUN:")
print("  result = run_step4()")
print("="*80)

Main execution function loaded

STEP 4 INITIALIZED - Ready to build recommendations

TO RUN:
  result = run_step4()
